In [ ]:
autosave 0

# Unification

This notebook implements the algorithm of Martelli and Montanari for the unification of terms.

## Utility Functions

In [ ]:
import folParser as fp

In [ ]:
def parseTerm(s):
    parser = fp.LogicParser(s)
    return parser.parse()

The method $\texttt{apply}(t, \sigma)$ takes a term $t$ and a substitution $\sigma$ and computes the term $t\sigma$, i.e. it applies the substitution $\sigma$ to the term $t$.  The substitution $\sigma$ is represented as a dictionary.  Assume that $\sigma = \bigl[ x_1 \mapsto t_1, \cdots, x_n \mapsto t_n \bigr]$.  Then $t\sigma$ is defined by induction on $t$:
<ol>
<li>If $t$ is a variable, there are two cases when defining $t\sigma$:</li>
  <ol>
  <li>$t = x_i$ for an $i\in\{1,\cdots,n\}$.  Then we define  
      $$ x_i\sigma := t_i. $$
      </li>
  <li>$t = y$ where $y\in\mathcal{V}$, but $y \not\in \{x_1,\cdots,x_n\}$. Then we define   
      $$ y\sigma := y.$$</li>
  </ol>
<li>Otherwise we must have $t = f(s_1,\cdots,s_m)$. Then we define: 
      $$f(s_1, \cdots, s_m)\sigma := f(s_1\sigma, \cdots, s_m\sigma). $$
      </li>
</ol>

In [ ]:
def apply(t, σ):
    "Apply the substitution σ to the term t."
    if isinstance(t, str): # t is a variable
        if t in σ:
            return σ[t]
        else:
            return t
    else: 
        f  = t[0]
        ts = t[1:]
        return (f,) + tuple(apply(s, σ) for s in ts)

In [ ]:
t = parseTerm('F(x,H(y,x),G(z))')
t

In [ ]:
s1 = parseTerm('G(z)')
s2 = parseTerm('H(u, v)')
σ = { 'x': s1, 'y': s2 }
σ

In [ ]:
apply(t, σ)

If  $\sigma = \big[ x_1 \mapsto s_1, \cdots, x_m \mapsto s_m \big]$ and
$\tau = \big[ y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$ 
are two substitutions that are <em>non-overlapping</em>, i.e. such that $\texttt{dom}(\sigma) \cap \texttt{dom}(\tau) = \{\}$ holds,
then we define the <em>composition</em> $\sigma\tau$ of $\sigma$ and $\tau$ as follows:
$$\sigma\tau := \big[ x_1 \mapsto s_1\tau, \cdots, x_m \mapsto s_m\tau,\; y_1 \mapsto t_1, \cdots, y_n \mapsto t_n \big]$$
The function $\texttt{compose}(\sigma, \tau)$ takes two non-overlapping substitutions and computes the composition $\sigma\tau$.

In [ ]:
def applySet(S, σ):
    return { (apply(t1, σ), apply(t2, σ)) for (t1, t2) in S }

In [ ]:
def compose(σ, τ):
    Result = { x: apply(s, τ) for (x, s) in σ.items() }
    Result.update(τ)
    return Result

In [ ]:
τ = { 'z': s1, 'u': s2 }

In [ ]:
compose(σ, τ)

The function $\texttt{occurs}(x, t)$ checks whether the variable $x$ occurs in the term $t$.

In [ ]:
def occurs(x, t):
    if x == t:
        return True
    if isinstance(t, str):
        return False
    return any(occurs(x, arg) for arg in t[1:])

In [ ]:
t

In [ ]:
occurs('u', t)

In [ ]:
occurs('x', t)

## The Algorithm of Martelli and Montanari

Given two terms $s$ and $t$, the function $\texttt{unify}(s, t)$ computes the *most general unifier* of $s$ and $t$.

In [ ]:
def unify(s, t):
    return solve({(s, t)}, {})

Given a set of *syntactical equations* $E$ and a substitution $\sigma$, the function $\texttt{solve}(E, \sigma)$ applies the rules of Martelli and Montanari to solve $E$.

In [ ]:
def solve(E, σ):
    while E != set():
        print(E, σ)
        (s, t) = E.pop()
        if s == t:
            continue
        if isinstance(s, str):
            if occurs(s, t):
                return None
            else:
                applySet(E, { s: t })
                σ = compose(σ, { s: t })
        elif isinstance(t, str):
            E.add((t, s))
        else:
            f    , g     = s[0]      , t[0]
            sArgs, tArgs = s[1:]     , t[1:]
            m    , n     = len(sArgs), len(tArgs)
            if f != g or m != n:
                return None
            else:
                E |= { (sArgs[i], tArgs[i]) for i in range(m) }
    return σ

In [ ]:
t1 = parseTerm('P(x1,F(x4))')
t2 = parseTerm('P(x2,x3)')
μ = unify(t1, t2)
μ